In [3]:
import pandas as pd
import numpy as np

from rl_lib.swiss_round.environment import SwissRoundEnv
from rl_lib.swiss_round.agent import DQNAgent

%reload_ext autoreload
%autoreload 2

# Utils 

In [4]:
def probability_tables(team_strengths, max_draw_probability):
    index = range(len(team_strengths))
    wps = []
    dps = []
    lps = []
    for ts1 in team_strengths :
        twps = []
        tdps = []
        tlps = []
        for ts2 in team_strengths :
            strength_diff = ts1 - ts2
            tmp_win_prob = 1 / (1 + np.exp(-strength_diff))
            tmp_loss_prob = 1 / (1 + np.exp(+strength_diff))
            tmp_draw_prob = max_draw_probability * np.exp(-abs(strength_diff))
            # Softmax
            win_prob = tmp_win_prob / (tmp_win_prob + tmp_draw_prob + tmp_loss_prob)
            draw_prob = tmp_draw_prob / (tmp_win_prob + tmp_draw_prob + tmp_loss_prob)
            loss_prob = tmp_loss_prob / (tmp_win_prob + tmp_draw_prob + tmp_loss_prob)  
            
            twps.append(win_prob)
            tdps.append(draw_prob)
            tlps.append(loss_prob)
        wps.append(twps)
        dps.append(tdps)
        lps.append(tlps)
    return pd.DataFrame(wps, index=index, columns = index), pd.DataFrame(dps, index=index, columns = index),pd.DataFrame(lps, index=index, columns = index)        

In [5]:
def check_probability(team_strengths, max_draw_probability):
    wp, dp, lp = probability_tables(team_strengths=team_strengths, max_draw_probability=max_draw_probability)
    df = wp+dp+lp
    values_array = df.to_numpy()
    target_array = np.full_like(values_array, 1)

    return np.allclose(values_array, target_array, rtol=10e-5, atol=10e-8)

# Environement

In [6]:
n_teams = 18
max_str = 4
threshold_ranks = [4,12]
agent_id = threshold_ranks[-1] 
team_strengths = np.linspace(max_str,0,n_teams)
#team_strengths = [4 * 0.9 ** i for i in range(n_teams)]
mdp = 0.5

print(np.array(team_strengths).round(2))
wp, dp, lp = probability_tables(team_strengths=team_strengths, max_draw_probability=mdp)
pd.concat([wp.loc[[agent_id]].rename(index={agent_id:'Win'}),
           dp.loc[[agent_id]].rename(index={agent_id:'Draw'}),
           lp.loc[[agent_id]].rename(index={agent_id:'Loss'}),
           ]).T.round(2)

[4.   3.76 3.53 3.29 3.06 2.82 2.59 2.35 2.12 1.88 1.65 1.41 1.18 0.94
 0.71 0.47 0.24 0.  ]


,Win,Draw,Loss
0,0.05,0.03,0.92
1,0.07,0.04,0.90
2,0.08,0.05,0.87
3,0.10,0.06,0.84
4,0.12,0.07,0.81
5,0.15,0.09,0.76
6,0.17,0.11,0.72
7,0.20,0.13,0.66
8,0.23,0.16,0.60
9,0.27,0.20,0.54


In [7]:
n_rounds = 6
bonus_points = [30,20]
#Agent_id just below last threshold
n_baselines_simu = 2000
name = f'lin_{max_str}'
print(f"Valid probability set-up : {check_probability(team_strengths, mdp)}")

env = SwissRoundEnv(
    n_teams=n_teams,
    n_rounds=n_rounds,
    name=name,
    team_strengths=team_strengths,
    threshold_ranks=threshold_ranks,
    bonus_points=bonus_points,
    agent_id=agent_id,
    max_draw_probability=mdp
)

Valid probability set-up : True


#### Detailed tables

In [8]:
wp.round(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87,0.90,0.92,0.93,0.95,0.96,0.97,0.97
1,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87,0.90,0.92,0.93,0.95,0.96,0.97
2,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87,0.90,0.92,0.93,0.95,0.96
3,0.27,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87,0.90,0.92,0.93,0.95
4,0.23,0.27,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87,0.90,0.92,0.93
5,0.20,0.23,0.27,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87,0.90,0.92
6,0.17,0.20,0.23,0.27,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87,0.90
7,0.15,0.17,0.20,0.23,0.27,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87
8,0.12,0.15,0.17,0.20,0.23,0.27,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84
9,0.10,0.12,0.15,0.17,0.20,0.23,0.27,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81


In [9]:
dp.round(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05,0.04,0.03,0.02,0.02,0.01,0.01,0.01
1,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05,0.04,0.03,0.02,0.02,0.01,0.01
2,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05,0.04,0.03,0.02,0.02,0.01
3,0.20,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05,0.04,0.03,0.02,0.02
4,0.16,0.20,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05,0.04,0.03,0.02
5,0.13,0.16,0.20,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05,0.04,0.03
6,0.11,0.13,0.16,0.20,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05,0.04
7,0.09,0.11,0.13,0.16,0.20,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05
8,0.07,0.09,0.11,0.13,0.16,0.20,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06
9,0.06,0.07,0.09,0.11,0.13,0.16,0.20,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07


In [10]:
lp.round(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08,0.07,0.05,0.04,0.04,0.03,0.02,0.02
1,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08,0.07,0.05,0.04,0.04,0.03,0.02
2,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08,0.07,0.05,0.04,0.04,0.03
3,0.54,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08,0.07,0.05,0.04,0.04
4,0.60,0.54,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08,0.07,0.05,0.04
5,0.66,0.60,0.54,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08,0.07,0.05
6,0.72,0.66,0.60,0.54,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08,0.07
7,0.76,0.72,0.66,0.60,0.54,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08
8,0.81,0.76,0.72,0.66,0.60,0.54,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10
9,0.84,0.81,0.76,0.72,0.66,0.60,0.54,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12


In [11]:
wp+dp+lp

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


# Simulations

### Logs

In [12]:
display_cols = ['exp_id', 'env_name', 'n_episodes','train_epochs','lr','use_lr_scheduler','gradient_clipping','epsilon_end','epsilon_decay','activation', 'layers', 'dropout', 'avg_test_rewards',
       'std_test_rewards', 'avg_test_gambits', 'std_test_gambits']

In [26]:
cpdf = pd.read_csv('../logs/calmip_logs.csv', index_col=0)
locdf = pd.read_csv('../logs/exp_logs.csv', index_col=0)
ldf = pd.concat([cpdf,locdf]).reset_index(drop=True)
ldf[ldf['env_name']==name].sort_values(by='avg_test_rewards', ascending=False)[display_cols]

,exp_id,env_name,n_episodes,train_epochs,lr,use_lr_scheduler,gradient_clipping,epsilon_end,epsilon_decay,activation,layers,dropout,avg_test_rewards,std_test_rewards,avg_test_gambits,std_test_gambits
18,cp_15,lin_4,50000,1.0,0.0005,False,1000000.0,0.04,0.9995,relu,256_64,0.3,17.1000,13.764738,0.0550,0.232325
11,cp_4,lin_4,32000,1.0,0.0010,False,1000000.0,0.02,0.9995,relu,256_64,0.3,16.9570,13.201597,0.0500,0.233452
10,cp_3,lin_4,32000,1.0,0.0010,False,1000000.0,0.02,0.9995,relu,256_128_64,0.3,16.8940,13.216572,0.1205,0.376802
17,cp_14,lin_4,50000,1.0,0.0005,False,1000000.0,0.02,0.9995,relu,256_64,0.3,16.8625,13.562802,0.0140,0.117490
12,cp_9,lin_4,50000,1.0,0.0010,False,1000000.0,0.02,0.9995,relu,128_64,0.3,16.8550,13.125166,0.1030,0.358317
28,loc_10,lin_4,20000,1.0,0.0005,False,1000000.0,0.01,0.9995,relu,128_32,0.3,16.8310,13.173437,0.0775,0.288953
16,cp_13,lin_4,50000,1.0,0.0010,False,1000000.0,0.02,0.9995,relu,256_64,0.5,16.8110,13.515964,0.0225,0.158094
21,loc_3,lin_4,32000,1.0,0.0003,False,1000000.0,0.05,0.9990,relu,64_64,0.2,16.7295,13.797910,0.0205,0.148593
14,cp_11,lin_4,50000,1.0,0.0010,False,1000000.0,0.02,0.9995,relu,128_64,0.5,16.6960,13.267275,0.0430,0.226166
20,loc_2,lin_4,20000,1.0,0.0005,False,1000000.0,0.01,0.9995,relu,256_128_64,0.3,16.6870,13.713462,0.0705,0.267450


### Baselines Simulations

In [12]:
simulation_wa = env.simulate_n_tournaments(n_baselines_simu,
                                           n_cores = 32, 
                                           policy = 'win_all',
                                           display_results=True)
baseline_reward_wa = simulation_wa.loc[agent_id,'Avg_Reward'] 
baseline_std_wa = simulation_wa.loc[agent_id,'Std_Reward'] 
print(f"Baseline WinAll average reward = {baseline_reward_wa:.1f} ± {baseline_std_wa:.1f}")
simulation_wa.loc[agent_id].round(2)

Simulating tournaments: 100%|██████████| 2000/2000 [00:09<00:00, 200.71it/s]



Simulation Results (from 1980 tournaments):
Team | Strength | Avg Points | Avg Rank | Avg Reward | Std Reward | Top-4 % | Top-12 % | 
----------------------------------------------------------------------------------------
   0 |     4.00 |      13.14 |     3.19 |      55.76 |      15.07 |  75.71% |  99.49% | 
   1 |     3.76 |      12.54 |     3.79 |      53.03 |      16.20 |  68.74% |  99.34% | 
   2 |     3.53 |      11.89 |     4.44 |      49.34 |      17.20 |  58.99% |  98.74% | 
   3 |     3.29 |      11.31 |     5.19 |      45.46 |      17.82 |  48.79% |  97.58% | 
   4 |     3.06 |      10.91 |     5.71 |      42.70 |      17.89 |  41.77% |  96.31% | 
   5 |     2.82 |      10.35 |     6.55 |      38.65 |      17.20 |  31.11% |  94.85% | 
   6 |     2.59 |       9.75 |     7.43 |      34.81 |      16.41 |  22.53% |  91.52% | 
   7 |     2.35 |       9.21 |     8.31 |      31.47 |      16.12 |  16.87% |  86.01% | 
   8 |     2.12 |       8.78 |     9.02 |      28.48 |      15.0

Team          12.00
Strength       1.18
Avg_Points     6.65
Avg_Rank      12.54
Avg_Reward    16.36
Std_Reward    13.92
Top-4 %        0.03
Top-12 %       0.44
Name: 12, dtype: float64

In [13]:
simulation_lf = env.simulate_n_tournaments(n_baselines_simu,n_cores = 32, policy = 'lose_first',display_results=False)
baseline_reward_lf = simulation_lf.loc[agent_id,'Avg_Reward'] 
baseline_std_lf = simulation_lf.loc[agent_id,'Std_Reward'] 
print(f"Baseline Lose First average reward = {baseline_reward_lf:.1f} ± {baseline_std_lf:.1f}")
simulation_lf.loc[agent_id].round(2)

Simulating tournaments: 100%|██████████| 2000/2000 [00:10<00:00, 199.05it/s]

Baseline Lose First average reward = 14.4 ± 12.7


Team          12.00
Strength       1.18
Avg_Points     6.37
Avg_Rank      13.13
Avg_Reward    14.36
Std_Reward    12.75
Top-4 %        0.01
Top-12 %       0.38
Name: 12, dtype: float64

### RL Agent

In [14]:
print(f"Baselines average reward : WinAll = {baseline_reward_wa:.1f} ± {baseline_std_wa:.1f}, "
      f"LoseFirst = {baseline_reward_lf:.1f} ± {baseline_std_lf:.1f}")

Baselines average reward : WinAll = 16.4 ± 13.9, LoseFirst = 14.4 ± 12.7


In [14]:
agent = DQNAgent(env,
                 hidden_dims=[128,32],
                 dropout= 0.3,
                 lr = 0.0005,
                 buffer_size=10000,
                 epsilon_decay=0.9995,
                 activation='tanh',
                 train_epochs=8,
                 max_grad_norm=1,
                 use_lr_scheduler=True,
                 n_train_episodes=6000,
                 n_test_episodes=2000)
agent.train()
agent.evaluate()


--- Training in progress ---


KeyboardInterrupt: 

### Verbosed simulation

In [22]:
# Simulate tournament
final_standings = env.simulate_tournament(verbose= True)

print("\nFinal standings (team_id, points, opponent_average):")
for rank, (team_id, points, reward, opp_avg,strength) in enumerate(final_standings, 1):

    print(f"Rank {rank}: Team {team_id} - Strength {strength:.2f} - Points: {points} (reward {reward}) - Opponent Avg: {opp_avg:.2f}")


--- Simulating round n°1 ---
Game : Team 8 (points : 0, strength : 2.12) vs Team 2 (points : 0, strength : 3.53) : Draw
Game : Team 5 (points : 0, strength : 2.82) vs Team 1 (points : 0, strength : 3.76) : Team 1 wins
Game : Team 12 (points : 0, strength : 1.18) vs Team 11 (points : 0, strength : 1.41) : Team 11 wins
Game : Team 17 (points : 0, strength : 0.00) vs Team 9 (points : 0, strength : 1.88) : Team 9 wins
Game : Team 0 (points : 0, strength : 4.00) vs Team 10 (points : 0, strength : 1.65) : Team 10 wins
Game : Team 3 (points : 0, strength : 3.29) vs Team 6 (points : 0, strength : 2.59) : Draw
Game : Team 15 (points : 0, strength : 0.47) vs Team 14 (points : 0, strength : 0.71) : Team 14 wins
Game : Team 13 (points : 0, strength : 0.94) vs Team 7 (points : 0, strength : 2.35) : Team 7 wins
Game : Team 4 (points : 0, strength : 3.06) vs Team 16 (points : 0, strength : 0.24) : Team 4 wins
--- Simulating round n°2 ---
Game : Team 1 (points : 3, strength : 3.76) vs Team 4 (points :